In [2]:
%load_ext tensorboard

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, concatenate, Flatten
from tensorflow.keras.optimizers import SGD
import pandas as pd
import numpy as np
from PIL import Image
from math import floor
import matplotlib.pyplot as plt
import datetime

# Clear logs from previous runs
# %rm -rf ../logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
NUM_INPUTS = 15
NUM_OUTPUTS = 11

# Load the dataset input and output
inputNums = np.loadtxt('../dataset/0_input.csv', delimiter=',') # (num training points, 13)
y = np.loadtxt('../dataset/0_output.csv', delimiter=',') # (num training points, 11)
X = np.zeros((len(inputNums), 480, 640, 1)) # (num training points, 480, 640)

# For each of the training examples there should be exactly 1 image
for i in range(0, len(inputNums)):
    image = Image.open('../dataset/img_' + str(i) + '.jpg') 
    imageArr = np.asarray(image).reshape(480, 640, 1) # Converts the image into a 2D 480*640 array
    X[i] = imageArr / 255


In [3]:
numerical_inputs = Input(shape=(NUM_INPUTS,))
image_inputs = Input(shape=(480, 640, 1))

# Process the image data first
x_image = Conv2D(32, (7, 7), strides=(4, 4), padding='same', activation='relu')(image_inputs)
x_image = Conv2D(64, (5, 5), strides=(4, 4), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Model(image_inputs, x_image)
x_image = Flatten()(x_image.output)

print(x_image.shape)
print(numerical_inputs.shape)

# Combine the latent space representation with the numerical input
combined = concatenate([x_image, numerical_inputs])
print(combined.shape)
x_combined = Dense(300, activation='relu')(combined)
x_combined = Dense(60, activation='relu')(x_combined)
outputs = Dense(11, activation='linear')(x_combined)

model = Model([image_inputs, numerical_inputs], outputs)

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit([X, inputNums], y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback])

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

(None, 5120)
(None, 15)
(None, 5135)
Epoch 1/5
 1/19 [>.............................] - ETA: 0s - loss: 6.4982 - mae: 1.2935WARNING:tensorflow:From /home/wenhan/miniconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
19/19 [==============================] - 7s 394ms/step - loss: 4.3427 - mae: 1.1983 - val_loss: 5.7431 - val_mae: 1.4426
Epoch 2/5
19/19 [==============================] - 7s 372ms/step - loss: 1.1032 - mae: 0.7337 - val_loss: 5.1639 - val_mae: 1.3969
Epoch 3/5
19/19 [==============================] - 9s 450ms/step - loss: 0.5843 - mae: 0.5323 - val_loss: 3.5931 - val_mae: 1.1986
Epoch 4/5
19/19 [==============================] - 9s 498ms/step - loss: 0.3894 - mae: 0.4376 - val_loss: 2.8618 - val_mae: 1.0984
Epoch 5/5
19/19 [==============================] - 10s 535ms/step - los

In [6]:
# Save the model
model.save('../models')

# Open Tensorboard (actually, run this on the command line on your Terminal)
# %tensorboard --logdir ../logs/fit

INFO:tensorflow:Assets written to: ../models/assets


In [ ]:
# Load the model
loaded_model = model_load('../models')
loaded_model.fit([X, inputNums], y, epochs=100, batch_size=32, validation_split=0.2)